In [1]:
from IPython.display import clear_output
# !pip install --no-index --no-deps /kaggle/input/predetectron-31-wheel/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torch-1.12.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torchvision-0.13.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl 
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/openmim-0.3.9-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmengine-0.7.4-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/addict-2.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/ai-jam-inference/yapf-0.40.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/ai-jam-inference/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/ai-jam-inference/setuptools-69.5.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/einops-0.6.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/mat4py-0.5.0-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/ordered_set-4.1.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/model_index-0.1.11-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/modelindex-0.0.2-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmpretrain/mmpretrain-1.0.0rc8-py2.py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/ai-jam-inference/ensemble_boxes-1.0.9-py3-none-any.whl
clear_output()

In [2]:
!pip install --no-index --no-deps /kaggle/input/ai-jam-inference/mmdet-3.3.0-py3-none-any.whl

clear_output()

In [3]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())
# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)
# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)
# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)

torch version: 1.12.1+cu116 cuda: True
mmdetection: 3.3.0
mmcv: 2.0.1
mmengine: 0.7.4


In [4]:
# We download the pre-trained checkpoints for inference and finetuning.
#!mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest /kaggle/working/checkpoints

In [5]:
!pip install --no-index \
            --find-links=/kaggle/input/prepare-yolo-env \
            ultralytics==8.3.169

clear_output()

In [6]:
import os
from pathlib import Path
from typing import List, Tuple
import math

import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
import cv2

# MMDetection imports
from mmdet.apis import DetInferencer

# YOLO and ensemble imports
from ultralytics import YOLO
from ensemble_boxes import weighted_boxes_fusion

/usr/local/lib/python3.10/dist-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# Global model configuration
MODEL_CONFIG = '/kaggle/input/ai-jam-inference/rtmdet_tiny_1xb4-20e_demine_test (1).py'
CHECKPOINT = '/kaggle/input/ai-jam-inference/best_coco_bbox_mAP_epoch_19_semifixedv2.pth'

# Multiple YOLO models
YOLO_MODEL_PATHS = [
    "/kaggle/input/demining-models/yolo11_onlylabeletrain_atleast1boxcrop_underfited/yolo11_onlylabeletrain_atleast1boxcrop_underfited/best.pt",
    "/kaggle/input/demining-models/yolo11_onlylabeletrain_atleast1boxcrop_middlefited/yolo11_onlylabeletrain_atleast1boxcrop_middlefited/last.pt",
    "/kaggle/input/demining-models/yolo11_onlylabeletrain_atleast1boxcrop_underfited/yolo11_onlylabeletrain_atleast1boxcrop_underfited/best.pt",
    "/kaggle/input/demining-models/yolo11_onlylabeletrain_atleast1boxcrop_underfited/yolo11_onlylabeletrain_atleast1boxcrop_underfited/last.pt"
]

DEVICE = 'cuda:0'
CONFIDENCE_THRESHOLD = 0.3
BATCH_SIZE = 16

# WBF Configuration
YOLO_ENSEMBLE_WEIGHT = 1.0  # Weight for YOLO ensemble in final fusion
MMDET_WEIGHT = 2.0          # Weight for MMDet in final fusion
WBF_IOU_THRESHOLD = 0.5
WBF_SKIP_BOX_THRESHOLD = 0.0001

# Class mapping for submission
CLASS_LABEL_MAPPING = {
    0: 0,  # anti_tank_mine -> 0
    1: 1,  # anti_personnel_mine -> 1  
    2: 2   # other_explosive -> 2
}

# Global model instances
_model_instance = None
_yolo_models = []

In [8]:


def get_model():
    """Get or initialize the MMDet model instance"""
    global _model_instance
    if _model_instance is None:
        print("Loading MMDetection model...")
        _model_instance = DetInferencer(MODEL_CONFIG, CHECKPOINT, DEVICE)
        print("MMDet model loaded successfully!")
    return _model_instance

def get_yolo_models():
    """Get or initialize all YOLO model instances"""
    global _yolo_models
    if not _yolo_models:
        print(f"Loading {len(YOLO_MODEL_PATHS)} YOLO models...")
        for i, path in enumerate(YOLO_MODEL_PATHS):
            print(f"  Loading YOLO model {i+1}/{len(YOLO_MODEL_PATHS)}: {Path(path).name}")
            _yolo_models.append(YOLO(path))
        print("All YOLO models loaded successfully!")
    return _yolo_models

def preprocess_image(image_path: str, target_size: int = 640) -> torch.Tensor:
    """
    Load, preprocess image and return 1CHW tensor 
    
    Args:
        image_path: Path to the image file
        target_size: Target size for the model input
    
    Returns:
        Preprocessed image tensor
    """
    # Load image
    image = Image.open(image_path).convert('RGB')
    
    # Resize image
    image = image.resize((target_size, target_size))
    
    # Convert to tensor and normalize
    image_tensor = torch.from_numpy(np.array(image)).float()
    image_tensor = image_tensor.permute(2, 0, 1) / 255.0  # HWC to CHW and normalize
    
    # Add batch dimension
    image_tensor = image_tensor.unsqueeze(0)
    
    return image_tensor.cuda()

def get_image_dimensions(image_path: str) -> Tuple[int, int]:
    """Get original image dimensions"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None, None
        height, width = img.shape[:2]
        return width, height
    except:
        return None, None

def normalize_bbox(bbox, img_width: int, img_height: int) -> Tuple[float, float, float, float]:
    """Convert absolute bbox to normalized center format"""
    x1, y1, x2, y2 = bbox
    
    # Calculate center and dimensions
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    width = x2 - x1
    height = y2 - y1
    
    # Normalize to [0, 1]
    cx_norm = center_x / img_width
    cy_norm = center_y / img_height
    w_norm = width / img_width
    h_norm = height / img_height
    
    # Clamp to [0, 1] range
    cx_norm = max(0, min(1, cx_norm))
    cy_norm = max(0, min(1, cy_norm))
    w_norm = max(0, min(1, w_norm))
    h_norm = max(0, min(1, h_norm))
    
    return cx_norm, cy_norm, w_norm, h_norm

def predict_batch_images(image_paths: List[str]) -> List[List[Tuple[int, float, float, float, float]]]:
    """
    Run ensemble batch prediction with multiple YOLOs + MMDet + WBF
    """
    try:
        # Get dimensions for valid images
        dimensions = []
        valid_paths = []
        valid_indices = []
        
        for i, image_path in enumerate(image_paths):
            img_width, img_height = get_image_dimensions(image_path)
            if img_width is not None and img_height is not None:
                dimensions.append((img_width, img_height))
                valid_paths.append(image_path)
                valid_indices.append(i)
        
        if not valid_paths:
            return [[] for _ in image_paths]
        
        # Get MMDet batch predictions
        mmdet_model = get_model()
        mmdet_results = mmdet_model(valid_paths, out_dir=None, no_save_vis=True, no_save_pred=True)
        
        # Get all YOLO batch predictions
        yolo_models = get_yolo_models()
        all_yolo_results = []
        for i, model in enumerate(yolo_models):
            print(f"Running YOLO model {i+1}/{len(yolo_models)} on batch...")
            yolo_results = model(valid_paths, conf=CONFIDENCE_THRESHOLD, verbose=False)
            all_yolo_results.append(yolo_results)
        
        # Process each image with ensemble
        all_predictions = [[] for _ in image_paths]
        
        for idx, (img_path, (img_width, img_height)) in enumerate(zip(valid_paths, dimensions)):
            # Extract MMDet predictions
            mmdet_pred = mmdet_results['predictions'][idx]
            mmdet_boxes, mmdet_scores, mmdet_labels = [], [], []
            
            if len(mmdet_pred['scores']) > 0:
                for bbox, score, label in zip(mmdet_pred['bboxes'], mmdet_pred['scores'], mmdet_pred['labels']):
                    if score >= CONFIDENCE_THRESHOLD:
                        x1, y1, x2, y2 = bbox
                        mmdet_boxes.append([x1/img_width, y1/img_height, x2/img_width, y2/img_height])
                        mmdet_scores.append(float(score))
                        mmdet_labels.append(int(label))
            
            # Extract all YOLO predictions
            all_yolo_boxes = []
            all_yolo_scores = []
            all_yolo_labels = []
            
            for yolo_results in all_yolo_results:
                yolo_boxes, yolo_scores, yolo_labels = [], [], []
                yolo_result = yolo_results[idx]
                
                if yolo_result.boxes is not None:
                    for box in yolo_result.boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        yolo_boxes.append([x1/img_width, y1/img_height, x2/img_width, y2/img_height])
                        yolo_scores.append(float(box.conf[0]))
                        yolo_labels.append(int(box.cls[0]))
                
                all_yolo_boxes.append(yolo_boxes)
                all_yolo_scores.append(yolo_scores)
                all_yolo_labels.append(yolo_labels)
            
            # Stage 1: Ensemble all YOLO models first
            if len(all_yolo_boxes) > 1:
                # Multiple YOLOs - ensemble them first
                yolo_weights = [1.0] * len(all_yolo_boxes)  # Equal weights for all YOLOs
                
                yolo_fused_boxes, yolo_fused_scores, yolo_fused_labels = weighted_boxes_fusion(
                    all_yolo_boxes, all_yolo_scores, all_yolo_labels,
                    weights=yolo_weights, 
                    iou_thr=WBF_IOU_THRESHOLD, 
                    skip_box_thr=WBF_SKIP_BOX_THRESHOLD
                )
            elif len(all_yolo_boxes) == 1:
                # Single YOLO - use directly
                yolo_fused_boxes = all_yolo_boxes[0]
                yolo_fused_scores = all_yolo_scores[0] 
                yolo_fused_labels = all_yolo_labels[0]
            else:
                # No YOLO predictions
                yolo_fused_boxes, yolo_fused_scores, yolo_fused_labels = [], [], []

            # Stage 2: Combine YOLO ensemble with MMDet
            final_boxes_list = [yolo_fused_boxes, mmdet_boxes]
            final_scores_list = [yolo_fused_scores, mmdet_scores]
            final_labels_list = [yolo_fused_labels, mmdet_labels]
            final_weights = [YOLO_ENSEMBLE_WEIGHT, MMDET_WEIGHT]
            
            fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
                final_boxes_list, final_scores_list, final_labels_list,
                weights=final_weights, 
                iou_thr=WBF_IOU_THRESHOLD, 
                skip_box_thr=WBF_SKIP_BOX_THRESHOLD
            )
            
            # Convert to submission format
            detected_objects = []
            for box, score, label in zip(fused_boxes, fused_scores, fused_labels):
                if score >= CONFIDENCE_THRESHOLD:
                    x1, y1, x2, y2 = box
                    cx = (x1 + x2) / 2
                    cy = (y1 + y2) / 2
                    w = x2 - x1
                    h = y2 - y1
                    
                    submission_label = CLASS_LABEL_MAPPING.get(int(label), int(label))
                    
                    detected_objects.append((
                        int(submission_label),
                        float(cx),
                        float(cy),
                        float(w),
                        float(h)
                    ))
            
            original_idx = valid_indices[idx]
            all_predictions[original_idx] = detected_objects
        
        return all_predictions
        
    except Exception as e:
        print(f"Batch ensemble processing failed: {str(e)}")
        return [[] for _ in image_paths]

def predict_single_image(image_path: str) -> List[Tuple[int, float, float, float, float]]:
    """
    Fallback single image prediction (not used with batch processing)
    """
    return []

def get_test_images(images_folder: str) -> List[str]:
    """
    Get all image files from the specified folder.
    
    Args:
        images_folder: Path to the folder containing images
    
    Returns:
        List of image file paths
    """
    image_extensions = {'.jpg', '.jpeg', '.png'}
    image_files = []
    
    for file_path in Path(images_folder).rglob('*'):
        if file_path.suffix.lower() in image_extensions:
            image_files.append(str(file_path))
    
    return sorted(image_files)

def generate_submission(
    images_folder: str = "/kaggle/input/uadamage-demining-competition/test",
    output_path: str = "submission.csv",
    use_batch_processing: bool = True
):   
    """
    Generate submission CSV with multi-model ensemble predictions
    
    Args:
        images_folder: Path to test images folder
        output_path: Output CSV file path
        use_batch_processing: Whether to use batch processing for speed
    """
    print(f"🔍 Scanning images in: {images_folder}")
    image_files = get_test_images(images_folder)
    
    if not image_files:
        raise ValueError(f"No image files found in {images_folder}")
    
    print(f"📁 Found {len(image_files)} images")
    print(f"⚙️ Using batch processing: {use_batch_processing}")
    print(f"📦 Batch size: {BATCH_SIZE}")
    print(f"🎯 Confidence threshold: {CONFIDENCE_THRESHOLD}")
    print(f"🤖 Using {len(YOLO_MODEL_PATHS)} YOLO models -> YOLO ensemble -> MMDetection with WBF")
    print(f"⚖️ Final weights: [YOLO_ensemble: {YOLO_ENSEMBLE_WEIGHT}, MMDet: {MMDET_WEIGHT}]")
    
    # Prepare data for CSV
    rows = []
    row_id = 1
    
    if use_batch_processing and len(image_files) > 1:
        # Process images in batches
        num_batches = math.ceil(len(image_files) / BATCH_SIZE)
        print(f"🚀 Processing {len(image_files)} images in {num_batches} batches...")
        
        for batch_idx in tqdm(range(num_batches), desc="Processing multi-model ensemble batches"):
            # Get batch of images
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, len(image_files))
            batch_image_files = image_files[start_idx:end_idx]
            
            # Run batch prediction
            batch_predictions = predict_batch_images(batch_image_files)
            
            # Process results for this batch
            for image_path, predictions in zip(batch_image_files, batch_predictions):
                # Extract scene ID from filename (remove extension)
                scene_id = Path(image_path).stem
                
                if predictions:
                    # Add predictions for this scene
                    for label, cx, cy, w, h in predictions:
                        rows.append({
                            'id': row_id,
                            'scene': scene_id,
                            'label': label,
                            'cx': round(cx, 6),
                            'cy': round(cy, 6),
                            'w': round(w, 6),
                            'h': round(h, 6)
                        })
                        row_id += 1
                else:
                    # No predictions - add empty scene marker
                    rows.append({
                        'id': row_id,
                        'scene': scene_id,
                        'label': -1,
                        'cx': -1,
                        'cy': -1,
                        'w': -1,
                        'h': -1
                    })
                    row_id += 1
    else:
        # Process images one by one (fallback)
        print("🔄 Processing images individually...")
        for image_path in tqdm(image_files, desc="Processing images"):
            # Extract scene ID from filename (remove extension)
            scene_id = Path(image_path).stem
            
            # Run prediction
            predictions = predict_single_image(image_path)
            
            if predictions:
                # Add predictions for this scene
                for label, cx, cy, w, h in predictions:
                    rows.append({
                        'id': row_id,
                        'scene': scene_id,
                        'label': label,
                        'cx': round(cx, 6),
                        'cy': round(cy, 6),
                        'w': round(w, 6),
                        'h': round(h, 6)
                    })
                    row_id += 1
            else:
                # No predictions - add empty scene marker
                rows.append({
                    'id': row_id,
                    'scene': scene_id,
                    'label': -1,
                    'cx': -1,
                    'cy': -1,
                    'w': -1,
                    'h': -1
                })
                row_id += 1

    # Create DataFrame and save to CSV
    df = pd.DataFrame(rows)
    
    # Validate submission format
    print("✅ Validating submission format...")
    
    # Check required columns
    required_cols = ['id', 'scene', 'label', 'cx', 'cy', 'w', 'h']
    assert all(col in df.columns for col in required_cols), "Missing required columns"
    
    # Check data types and ranges
    df['id'] = df['id'].astype(int)
    df['label'] = df['label'].astype(int)
    df['cx'] = df['cx'].astype(float)
    df['cy'] = df['cy'].astype(float)
    df['w'] = df['w'].astype(float)
    df['h'] = df['h'].astype(float)
    
    # Check coordinate ranges
    coord_cols = ['cx', 'cy', 'w', 'h']
    for col in coord_cols:
        non_empty = df[df[col] != -1][col]
        if len(non_empty) > 0:
            assert non_empty.min() >= 0 and non_empty.max() <= 1, f"{col} values out of range [0,1]"
    
    # Check label values
    valid_labels = {-1, 0, 1, 2}
    assert set(df['label'].unique()).issubset(valid_labels), "Invalid label values found"
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    
    print(f"💾 Solution saved to: {output_path}")
    print(f"📊 Total predictions: {len(df)}")
    
    # Print statistics
    detections = df[df['label'] != -1]
    empty_scenes = df[df['label'] == -1]
    
    print(f"\\n📈 Submission Statistics:")
    print(f"   Total rows: {len(df)}")
    print(f"   Unique scenes: {df['scene'].nunique()}")
    print(f"   Detections: {len(detections)}")
    print(f"   Empty scenes: {len(empty_scenes)}")
    print(f"   Detection rate: {((df['scene'].nunique() - len(empty_scenes)) / df['scene'].nunique() * 100):.1f}%")
    
    if len(detections) > 0:
        label_counts = detections['label'].value_counts().sort_index()
        print(f"\\n🏷️ Class Distribution:")
        class_names = {0: "Anti-tank mines", 1: "Anti-personnel mines", 2: "Other explosives"}
        for label, count in label_counts.items():
            print(f"   Class {label} ({class_names.get(label, 'Unknown')}): {count}")
    
    print(f"\\n🎯 Ready for Kaggle submission!")
    return df



In [9]:
# Main execution
if __name__ == "__main__":
    # Generate submission with multi-model ensemble batch processing
    df = generate_submission(
        use_batch_processing=True
    )

🔍 Scanning images in: /kaggle/input/uadamage-demining-competition/test
📁 Found 4 images
⚙️ Using batch processing: True
📦 Batch size: 16
🎯 Confidence threshold: 0.3
🤖 Using 4 YOLO models -> YOLO ensemble -> MMDetection with WBF
⚖️ Final weights: [YOLO_ensemble: 1.0, MMDet: 2.0]
🚀 Processing 4 images in 1 batches...


Processing multi-model ensemble batches:   0%|          | 0/1 [00:00<?, ?it/s]

Loading MMDetection model...
Loads checkpoint by local backend from path: /kaggle/input/ai-jam-inference/best_coco_bbox_mAP_epoch_19_semifixedv2.pth
07/27 12:08:50 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
07/27 12:08:50 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


Output()

MMDet model loaded successfully!


/usr/local/lib/python3.10/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming 
release, it will be required to pass the indexing argument. (Triggered internally at  
../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Loading 4 YOLO models...
  Loading YOLO model 1/4: best.pt
  Loading YOLO model 2/4: last.pt
  Loading YOLO model 3/4: best.pt
  Loading YOLO model 4/4: last.pt
All YOLO models loaded successfully!
Running YOLO model 1/4 on batch...
Running YOLO model 2/4 on batch...
Running YOLO model 3/4 on batch...
Running YOLO model 4/4 on batch...


Processing multi-model ensemble batches: 100%|██████████| 1/1 [00:07<00:00,  7.94s/it]

✅ Validating submission format...
💾 Solution saved to: submission.csv
📊 Total predictions: 4
\n📈 Submission Statistics:
   Total rows: 4
   Unique scenes: 4
   Detections: 1
   Empty scenes: 3
   Detection rate: 25.0%
\n🏷️ Class Distribution:
   Class 0 (Anti-tank mines): 1
\n🎯 Ready for Kaggle submission!


In [10]:
pd.read_csv("/kaggle/working/submission.csv")

,id,scene,label,cx,cy,w,h
0,1,000eda45a4f594e86867516ffea98533,-1,-1.000000,-1.000000,-1.000000,-1.000000
1,2,0d5849518ef6d6692cb63435aa50edaa,-1,-1.000000,-1.000000,-1.000000,-1.000000
2,3,0eb1a9e6e37eb7413cfe29f0e9d3b69f,0,0.852081,0.843308,0.135244,0.313385
3,4,0eb1e6262e21225e1fcdd8a6198af484,-1,-1.000000,-1.000000,-1.000000,-1.000000
